In [199]:
# Bibliotecas 
import pandas as pd
import datetime as dt
import re
pd.set_option('display.max_rows', None)

car=pd.read_csv('car_data.csv') # dataset com os preços dos carros novos (tmb tem velhos)

In [ ]:
projet=pd.read_csv('train.csv') # dataset do projeto
projet.head()
projet.info()

In [217]:
car.head()

,Unnamed: 0,Model,Year,Status,Mileage,Price,MSRP,brand,model
0,0,Acura TLX A-Spec,2022,New,Not available,"$49,445","MSRP $49,445",Acura,TLX A-Spec
1,1,Acura RDX A-Spec,2023,New,Not available,"$50,895",Not specified,Acura,RDX A-Spec
2,2,Acura TLX Type S,2023,New,Not available,"$57,745",Not specified,Acura,TLX Type S
3,3,Acura TLX Type S,2023,New,Not available,"$57,545",Not specified,Acura,TLX Type S
4,4,Acura MDX Sport Hybrid 3.0L w/Technology Package,2019,Used,"32,675 mi.","$40,990",$600 price drop,Acura,MDX Sport Hybrid 3.0L w/Technology Package


In [ ]:
car.info()

### colocar as colunas 'brand' e 'model' como no dataset do projeto

In [202]:
car['Model'] = car['Model'].str[4:] 

In [203]:
car['brand'] = car['Model'].str.split().str[0]
car['model'] = car['Model'].str.split(n=1).str[1]

In [204]:
New_Prices = car[car['Status'] == 'New'] # pegar nos carros novos

### transformar o nome da coluna do ano do carro para combinar com o dataset do projeto


In [205]:
New_Prices.rename(columns={'Year': 'model_year'}, inplace=True)

C:\Users\elisa\AppData\Local\Temp\ipykernel_5320\481542589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Prices.rename(columns={'Year': 'model_year'}, inplace=True)


In [139]:
# New_Prices = New_Prices.drop(columns=['Unnamed: 0', 'Model', 'Status', 'MSRP', 'Mileage'])
New_Prices.head()
New_Prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47852 entries, 0 to 115760
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   model_year  47852 non-null  int64 
 1   Price       47852 non-null  object
 2   brand       47852 non-null  object
 3   model       47852 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


### transformar o preço para inteiro e remover carateres inúteis

In [206]:
New_Prices['Price'] = New_Prices['Price'].str.replace('$', '').str.replace(',', '')

C:\Users\elisa\AppData\Local\Temp\ipykernel_5320\1486477273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Prices['Price'] = New_Prices['Price'].str.replace('$', '').str.replace(',', '')


In [207]:
New_Prices['Price'] = pd.to_numeric(New_Prices['Price'], errors='coerce')
New_Prices.dropna(subset=['Price'], inplace=True)
New_Prices['Price'] = New_Prices['Price'].astype(int)


C:\Users\elisa\AppData\Local\Temp\ipykernel_5320\289901451.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Prices['Price'] = pd.to_numeric(New_Prices['Price'], errors='coerce')
C:\Users\elisa\AppData\Local\Temp\ipykernel_5320\289901451.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Prices.dropna(subset=['Price'], inplace=True)
C:\Users\elisa\AppData\Local\Temp\ipykernel_5320\289901451.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

### fazer a média para carros da mesma marca e ano mas com preços diferentes

In [208]:
mean_new_car_prices = New_Prices.groupby(['brand', 'model', 'model_year'])['Price'].mean().reset_index()
mean_new_car_prices.head()
mean_new_car_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   brand       1474 non-null   object 
 1   model       1474 non-null   object 
 2   model_year  1474 non-null   int64  
 3   Price       1474 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 46.2+ KB


### juntar dataset projeto com o preço dos carros novos

In [209]:
merged_df = pd.merge(projet, mean_new_car_prices, on=['brand', 'model', 'model_year'], how='inner')

In [175]:
merged_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Price
0,Lexus,LC 500 Base,2022,"2,071 mi.",Gasoline,5.0L V8 32V PDI DOHC,Automatic,Nightfall Mica,Caramel,None reported,NaN,99950,110938.333333
1,Acura,MDX Technology,2022,"15,895 mi.",Gasoline,3.5L V6 24V GDI SOHC,10-Speed Automatic,Majestic Black Pearl,Parchment,None reported,NaN,47950,55673.571429
2,Ford,Explorer Limited,2022,"17,500 mi.",Hybrid,318.0HP 3.3L V6 Cylinder Engine Gas/Electric H...,10-Speed A/T,White,Black,None reported,Yes,46000,52050.652174
3,INFINITI,QX80 SENSORY,2022,"26,520 mi.",Gasoline,400.0HP 5.6L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,Brown,At least 1 accident or damage reported,Yes,64000,84444.444444
4,Dodge,Charger SRT Hellcat Widebody,2023,"5,000 mi.",Gasoline,717.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Gray,Black,None reported,Yes,110000,96864.416667


In [210]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         204 non-null    object 
 1   model         204 non-null    object 
 2   model_year    204 non-null    int64  
 3   milage        204 non-null    object 
 4   fuel_type     188 non-null    object 
 5   engine        204 non-null    object 
 6   transmission  204 non-null    object 
 7   ext_col       204 non-null    object 
 8   int_col       204 non-null    object 
 9   accident      197 non-null    object 
 10  clean_title   153 non-null    object 
 11  price         204 non-null    int64  
 12  Price         204 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 20.8+ KB


### não precisa disto, era só para verificar que tem marcas de carros do projeto que não estão no dataset novo

In [211]:
merged_result = pd.merge(projet, mean_new_car_prices, on=['brand', 'model', 'model_year'], how='left', indicator=True)

In [212]:
not_in_merged = merged_result[merged_result['_merge'] == 'left_only']
# Drop the indicator column if needed
# not_in_merged = not_in_merged.drop('_merge', axis=1)

In [213]:
not_in_merged['brand'].unique()

array(['Jeep', 'Toyota', 'Subaru', 'Lexus', 'Ford', 'Nissan', 'MINI',
       'Mazda', 'Mercedes-Benz', 'Dodge', 'Tesla', 'Audi', 'Lincoln',
       'Volkswagen', 'INFINITI', 'RAM', 'Hyundai', 'Acura', 'Rolls-Royce',
       'Jaguar', 'Porsche', 'Aston', 'Maserati', 'Land', 'BMW', 'Honda',
       'Kia', 'GMC', 'Chrysler', 'Buick', 'Cadillac', 'Chevrolet',
       'Rivian', 'Pontiac', 'Hummer', 'Mitsubishi', 'Volvo', 'Genesis',
       'Bentley', 'Alfa', 'Lamborghini', 'Ferrari', 'Lotus', 'McLaren',
       'Saturn', 'Saab', 'Lucid', 'FIAT', 'Polestar', 'Scion', 'Maybach',
       'smart', 'Bugatti', 'Mercury', 'Suzuki', 'Karma'], dtype=object)

In [196]:
condition = (not_in_merged['brand'] == 'Toyota') 
filtered_df = not_in_merged.loc[condition]
filtered_df.head()
# merged_df.info()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Price,_merge
1,Toyota,Highlander XLE,2015,"68,575 mi.",Gasoline,270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Silver,Gray,At least 1 accident or damage reported,Yes,26000,NaN,left_only
2,Toyota,Camry Hybrid XLE,2018,"69,641 mi.",Hybrid,208.0HP 2.5L 4 Cylinder Engine Gas/Electric Hy...,CVT Transmission,White,Black,At least 1 accident or damage reported,Yes,23995,NaN,left_only
12,Toyota,Camry Hybrid Base,2007,"53,249 mi.",Hybrid,147.0HP 2.4L 4 Cylinder Engine Gas/Electric Hy...,A/T,Silver,Gray,At least 1 accident or damage reported,Yes,9250,NaN,left_only
30,Toyota,Highlander XLE,2018,"74,000 mi.",Gasoline,295.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,27950,NaN,left_only
36,Toyota,Tundra SR5,2020,"26,300 mi.",Gasoline,381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Gray,Gray,At least 1 accident or damage reported,Yes,46199,NaN,left_only
